# Tutorial
This tutorial explains how to :
- Prepare experiments for two models with different scalers and set up an experiments
- Train the model
- Compare model results thanks to tensorboard
- Generate validation data and perform unscaling process.

## Review of two convolutional network
First go to src\model\components\conv1d_surr.py and read the model component definition.

Then you can compare the conv1d_surr model with the conv1d_surr_nopca one.
for conv1d_surr_nopca no PCA is shall be used during pre-processing. src\model\components\conv1d_surr_nopca.py

Open now the files configs\model_net\conv1d_surr_nopca.yaml and configs\model_net\conv1d_surr.yaml.

User parameters to instanciate the models are defined inside them.

## Create new experiment files
Run the code below to create two new experiment files.
When running an experiment with train.py, you run the defaults parameters defined in the train.yaml file but you overide parameters specified in the experiment file.

In [2]:
import yaml
from yaml import CLoader as Loader

# create a dictionary with the parameters
header = '# @package _global_'
Document = """
defaults:
   - override /model_net: conv1d_surr.yaml
task_name: "tutorial"
preprocessing:
   perform_decomp : True
   
tags: ["surrogate", "conv1d", "PCA"]
"""

# The above experiment will use the 'conv1d_surr' model.
# It will outputs the results in the folder 'outputs\tutorial'.
# The default decomposition which is a PCA will be performed during pre-processing.
# The experiments tags, accessible in Tensorboard visualisation will be "surrogate", "conv1d", "PCA"

yaml_doc = yaml.load(Document, Loader=Loader)

# write the dictionary to a yaml file
with open('../configs/experiment/tuto_conv_pca.yaml', 'w') as f:
    f.write(header+'\n')
    yaml.dump(yaml_doc, f, default_flow_style=False)

In [3]:
with open('../configs/experiment/tuto_conv_nopca.yaml', 'w') as f :
    Document = """
    defaults:
        - override /model_net: conv1d_surr_nopca.yaml
    task_name: "tutorial"
    preprocessing:
        perform_decomp : False
    tags: ["surrogate", "conv1d", "NoPCA"]
    """
    yaml_doc = yaml.load(Document, Loader=Loader)
    f.write(header+'\n')
    yaml.dump(yaml_doc, f)

## Train both models
Now open a python terminal and run the code below in the root directory of the project :
```bash
python src/train.py experiment=tuto_conv_pca.yaml
```

You can see the logs in the terminal.
Full hydra output files are copied in a new folder of outputs\tutorial\runs.    
Note that you defined the tutorial folder by specifying the task_name: "tutorial" in the experiment yaml file.    

**You can order your experiments by task_name and model type to compare the models with tensorboard more easily.**



Now run the code below in the root directory of the project :
```bash
python src/train.py experiment=tuto_conv_nopca.yaml
```

## Compare models with tensorboard

When training is over, you can see the results in tensorboard by running the code below in the root directory of the project :
```bash
tensorboard --logdir outputs\tutorial\runs
```

Open a browser and go to :
http://localhost:6006/

Now open the time series tab and compare the results of the two models.
open the train card to see the training results and the validation card to see the validation results.
Now open the HPARAMS tab and compare the hyperparameters of the two models.

Scroll down the hyperparameters and activate tags and hp_metric.
Now you can identify which model use a PCA and compare the metrics of both models.

# Perform inference and unscaling
Now we will generate validation data and perform unscaling process.


In [7]:
import os
os.chdir('../src')


import subprocess

# Define the command to run
command = ["python", "surrogate_inference.py"]

# Execute the command within a subprocess
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

# Read and print the output dynamically
for line in process.stdout:
    print(line, end='')

# Wait for the process to finish
process.wait()

c:\Users\romain.ribault\Anaconda3\envs\dmltp\lib\site-packages\pkg_resources\__init__.py:122: PkgResourcesDeprecationWarning: -0.1.0- is an invalid version and will not be supported in a future release
  warnings.warn(
c:\Users\romain.ribault\Anaconda3\envs\dmltp\lib\site-packages\pkg_resources\__init__.py:122: PkgResourcesDeprecationWarning: -0.1.0- is an invalid version and will not be supported in a future release
  warnings.warn(
Traceback (most recent call last):
  File "surrogate_inference.py", line 231, in <module>
    load_env_file(f"env.yaml")
  File "c:\Users\romain.ribault\Documents\GitHub\torchydra\src\utils\load_env_file.py", line 5, in load_env_file
    with open(path, 'r') as stream:
FileNotFoundError: [Errno 2] No such file or directory: 'env.yaml'


1